<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Bayesian Personalized Ranking (BPR)

This notebook serves as an introduction to Bayesian Personalized Ranking (BPR) model for implicit feedback.  In this tutorial, we focus on learning the BPR model using matrix factorization approach, hence, the model is sometimes also named BPRMF.

The implementation of the model is from [Cornac](https://github.com/PreferredAI/cornac), which is a framework for recommender systems with a focus on models leveraging auxiliary data (e.g., item descriptive text and image, social network, etc).

## 0 Global Settings and Imports

In [1]:
import sys
import os
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


System version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
Cornac version: 1.14.2


In [4]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# top k items to recommend
TOP_K = 10

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 2

## 1 BPR Algorithm

### 1.1 Personalized Ranking from Implicit Feedback

The task of personalized ranking aims at providing each user a ranked list of items (recommendations).  This is very common in scenarios where recommender systems are based on implicit user behavior (e.g. purchases, clicks).  The available observations are only positive feedback where the non-observed ones are a mixture of real negative feedback and missing values.

One usual approach for item recommendation is directly predicting a preference score $\hat{x}_{u,i}$ given to item $i$ by user $u$.  BPR uses a different approach by using item pairs $(i, j)$ and optimizing for the correct ranking given preference of user $u$, thus, there are notions of *positive* and *negative* items.  The training data $D_S : U \times I \times I$ is defined as:

$$D_S = \{(u, i, j) \mid i \in I^{+}_{u} \wedge j \in I \setminus I^{+}_{u}\}$$

where user $u$ is assumed to prefer $i$ over $j$ (i.e. $i$ is a *positive item* and $j$ is a *negative item*).


### 1.2 Objective Function

From the Bayesian perspective, BPR maximizes the posterior probability over the model parameters $\Theta$ by optimizing the likelihood function $p(i >_{u} j | \Theta)$ and the prior probability $p(\Theta)$.

$$p(\Theta \mid >_{u}) \propto p(i >_{u} j \mid \Theta) \times p(\Theta)$$

The joint probability of the likelihood over all users $u \in U$ can be simplified to:

$$ \prod_{u \in U} p(>_{u} \mid \Theta) = \prod_{(u, i, j) \in D_S} p(i >_{u} j \mid \Theta) $$

The individual probability that a user $u$ prefers item $i$ to item $j$ can be defined as:

$$ p(i >_{u} j \mid \Theta) = \sigma (\hat{x}_{uij}(\Theta)) $$

where $\sigma$ is the logistic sigmoid:

$$ \sigma(x) = \frac{1}{1 + e^{-x}} $$

The preference scoring function $\hat{x}_{uij}(\Theta)$ could be an arbitrary real-valued function of the model parameter $\Theta$.  Thus, it makes BPR a general framework for modeling the relationship between triplets $(u, i, j)$ where different model classes like matrix factorization could be used for estimating $\hat{x}_{uij}(\Theta)$.

For the prior, one of the common pratices is to choose $p(\Theta)$ following a normal distribution, which results in a nice form of L2 regularization in the final log-form of the objective function.

$$ p(\Theta) \sim N(0, \Sigma_{\Theta}) $$

To reduce the complexity of the model, all parameters $\Theta$ are assumed to be independent and having the same variance, which gives a simpler form of the co-variance matrix $\Sigma_{\Theta} = \lambda_{\Theta}I$.  Thus, there are less number of hyperparameters to be determined.

The final objective of the maximum posterior estimator:

$$ J = \sum_{(u, i, j) \in D_S} \text{ln } \sigma(\hat{x}_{uij}) - \lambda_{\Theta} ||\Theta||^2 $$

where $\lambda_\Theta$ are the model specific regularization paramerters.


### 1.3 Learning with Matrix Factorization

#### Stochastic Gradient Descent

As the defined objective function is differentible, gradient descent based method for optimization is naturally adopted.  The gradient of the objective $J$ with respect to the model parameters:

$$
\begin{align}
\frac{\partial J}{\partial \Theta} & = \sum_{(u, i, j) \in D_S} \frac{\partial}{\partial \Theta} \text{ln} \ \sigma(\hat{x}_{uij}) - \lambda_{\Theta} \frac{\partial}{\partial \Theta} ||\Theta||^2 \\
& \propto \sum_{(u, i, j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot  \frac{\partial}{\partial \Theta} \hat{x}_{uij} - \lambda_{\Theta} \Theta
\end{align}
$$

Due to slow convergence of full gradient descent, we prefer using stochastic gradient descent to optimize the BPR model.  For each triplet $(u, i, j) \in D_S$, the update rule for the parameters:

$$ \Theta \leftarrow \Theta + \alpha \Big( \frac{e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot \frac{\partial}{\partial \Theta} \hat{x}_{uij} + \lambda_\Theta \Theta \Big) $$

#### Matrix Factorization for Preference Approximation

As mentioned earlier, the preference scoring function $\hat{x}_{uij}(\Theta)$ could be approximated by any real-valued function.  First, the estimator $\hat{x}_{uij}$ is decomposed into:

$$ \hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} $$

The problem of estimating $\hat{x}_{ui}$ is a standard collaborative filtering formulation, where matrix factorization approach has shown to be very effective.  The prediction formula can written as dot product between user feature vector $w_u$ and item feature vector $h_i$:

$$ \hat{x}_{ui} = \langle w_u , h_i \rangle = \sum_{f=1}^{k} w_{uf} \cdot h_{if} $$

The  derivatives of matrix factorization with respect to the model parameters are:

$$
\frac{\partial}{\partial \theta} \hat{x}_{uij} = 
\begin{cases}
    (h_{if} - h_{jf})  & \text{if } \theta = w_{uf} \\
    w_{uf}             & \text{if } \theta = h_{if} \\
    -w_{uf}            & \text{if } \theta = h_{jf} \\
    0                  & \text{else}
\end{cases}
$$

In theory, any kernel can be used to estimate $\hat{x}_{ui}$ besides the dot product $ \langle \cdot , \cdot \rangle $.  For example, k-Nearest-Neighbor (kNN) has also been shown to achieve good performance.

#### Analogies to AUC optimization

By optimizing the objective function of BPR model, we effectively maximizing [AUC](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) measurement.  To keep the notebook focused, please refer to the [paper](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf) for details of the analysis (Section 4.1.1).

## 2 Cornac implementation of BPR

BPR is implemented in the [Cornac](https://cornac.readthedocs.io/en/latest/index.html) framework as part of the model collections.
* Detailed documentations of the BPR model in Cornac can be found [here](https://cornac.readthedocs.io/en/latest/models.html#bayesian-personalized-ranking-bpr).
* Source codes of the BPR implementation is available on the Cornac Github repository, which can be found [here](https://github.com/PreferredAI/cornac/blob/master/cornac/models/bpr/recom_bpr.pyx).


## 3 Cornac BPR movie recommender


### 3.1 Load and split data

To evaluate the performance of item recommendation, we adopted the provided `python_random_split` tool for the consistency.  Data is randomly split into training and test sets with the ratio of 75/25.


Note that Cornac also cover different [built-in schemes](https://cornac.readthedocs.io/en/latest/eval_methods.html) for model evaluation.

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

100%|███████████████████████████████████████████████████████████████████████████████████████| 4.81k/4.81k [00:08<00:00, 590KB/s]


,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [5]:
# TODO
# 1. Rating df 생성
rating_data = "/opt/ml/input/data/train/train_ratings.csv"

raw_rating_df = pd.read_csv(rating_data)
raw_rating_df
raw_rating_df['rating'] = 1.0 # implicit feedback

raw_rating_df.drop(['time'],axis=1,inplace=True)

print("Raw rating df")
print(raw_rating_df)

# users = set(raw_rating_df.loc[:, 'user'])
# items = set(raw_rating_df.loc[:, 'item'])

# #2. Genre df 생성
# genre_data = "/opt/ml/input/data/train/genres.tsv"

# raw_genre_df = pd.read_csv(genre_data, sep='\t')
# raw_genre_df = raw_genre_df.drop_duplicates(subset=['item']) #item별 하나의 장르만 남도록 drop
# # 장르 clustering 재정의
# # print(raw_genre_df)

# genre_dict = {genre:i for i, genre in enumerate(set(raw_genre_df['genre']))}
# raw_genre_df['genre']  = raw_genre_df['genre'].map(lambda x : genre_dict[x]) #genre id로 변경
# print("Raw genre df - changed to id")
# print(raw_genre_df)


Raw rating df
           user   item  rating
0            11   4643     1.0
1            11    170     1.0
2            11    531     1.0
3            11    616     1.0
4            11   2140     1.0
...         ...    ...     ...
5154466  138493  44022     1.0
5154467  138493   4958     1.0
5154468  138493  68319     1.0
5154469  138493  40819     1.0
5154470  138493  27311     1.0

[5154471 rows x 3 columns]


In [6]:
raw_rating_df.columns = ['userID', 'itemID','rating']
raw_rating_df

,userID,itemID,rating
0,11,4643,1.0
1,11,170,1.0
2,11,531,1.0
3,11,616,1.0
4,11,2140,1.0
...,...,...,...
5154466,138493,44022,1.0
5154467,138493,4958,1.0
5154468,138493,68319,1.0
5154469,138493,40819,1.0


In [9]:
train = raw_rating_df
# test = python_random_split(raw_rating_df, 0.75)

In [7]:
train = raw_rating_df
test = raw_rating_df


### 3.2 Cornac Dataset

To work with models implemented in Cornac, we need to construct an object from [Dataset](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.dataset) class.

Dataset Class in Cornac serves as the main object that the models will interact with.  In addition to data transformations, Dataset provides a bunch of useful iterators for looping through the data, as well as supporting different negative sampling techniques.

In [8]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 31360
Number of items: 6807


### 3.3 Train the BPR model

The BPR has a few important parameters that we need to consider:

- `k`: controls the dimension of the latent space (i.e. the size of the vectors  $w_u$  and  $h_i$ ).
- `max_iter`: defines the number of iterations of the SGD procedure.
- `learning_rate`: controls the step size $\alpha$ in the gradient update rules.
- `lambda_reg`: controls the L2-Regularization $\lambda$ in the objective function.

Note that different values of `k` and `max_iter` will affect the training time.

We will here set `k` to 200, `max_iter` to 100, `learning_rate` to 0.01, and `lambda_reg` to 0.001. To train the model, we simply need to call the `fit()` method.

In [9]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [10]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 2/2 [00:09<00:00,  4.50s/it, correct=87.70%, skipped=4.42%]

Optimization finished!
Took 9.4526 seconds for training.


In [18]:
bpr.save('testmodelsave.h5')


BPR model is saved to testmodelsave.h5/BPR/2022-04-01_07-44-20-086430.pkl


'testmodelsave.h5/BPR/2022-04-01_07-44-20-086430.pkl'

In [12]:
import os

import tensorflow as tf
from tensorflow import keras

In [22]:
import pickle

In [23]:
new_model = pickle.load(open('/opt/ml/Recommenders/testmodelsave.h5/BPR/2022-04-01_07-44-20-086430.pkl','rb'))

In [24]:
with Timer() as t:
    new_model.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 2/2 [00:09<00:00,  4.60s/it, correct=87.67%, skipped=4.41%]

Optimization finished!
Took 9.2405 seconds for training.


In [25]:
train_set

### 3.4 Prediction and Evaluation

Now that our model is trained, we can produce the ranked lists for recommendation.  Every recommender models in Cornac provide `rate()` and `rank()` methods for predicting item rated value as well as item ranked list for a given user.  To make use of the current evaluation schemes, we will through `predict()` and `predict_ranking()` functions inside `cornac_utils` to produce the predictions.

Note that BPR model is effectively designed for item ranking.  Hence, we only measure the performance using ranking metrics.

In [10]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 248.3323 seconds for prediction.


In [ ]:

# with Timer() as t:
#     seen_all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=False)
# print("Took {} seconds for prediction.".format(t))

In [26]:
len(all_predictions[all_predictions['itemID']==8961])

19503

In [11]:
temp = all_predictions

In [12]:
temp

,userID,itemID,prediction
5154471,11,8961,5.935120
5154472,11,1396,-0.469809
5154473,11,471,-2.465719
5154474,11,1042,0.387059
5154475,11,1947,-0.819334
...,...,...,...
213467515,138493,7753,-3.388236
213467516,138493,93422,-3.212936
213467517,138493,6519,-1.739218
213467518,138493,8830,-0.618888


In [13]:
ttmp = temp.sort_values(by=["prediction"]).groupby("userID")
ttmp


In [14]:
tttmp=pd.DataFrame(
    data={
        "userID": list(ttmp.groups.keys()),
        "itemID": list(ttmp.itemID.apply(list)),
        "prediction":list(ttmp.prediction.apply(list)),
    }
)

In [15]:
tttmp

,userID,itemID,prediction
0,11,"[2922, 2690, 91134, 7889, 4276, 107406, 110586...","[-8.578864097595215, -8.195862770080566, -7.61..."
1,14,"[69784, 3272, 65225, 3761, 481, 67508, 6868, 6...","[-7.967667579650879, -7.656135559082031, -7.48..."
2,18,"[1297, 8985, 108979, 2143, 83132, 8633, 37380,...","[-7.405815124511719, -7.285192012786865, -6.92..."
3,25,"[30712, 8752, 26203, 3634, 109578, 5046, 6550,...","[-6.363016128540039, -5.6773295402526855, -5.6..."
4,31,"[7486, 535, 1076, 2356, 52, 8044, 26704, 7396,...","[-6.797060489654541, -6.735404968261719, -6.34..."
...,...,...,...
31355,138473,"[5004, 4005, 5604, 4506, 26555, 5603, 6533, 46...","[-6.395918846130371, -6.344394207000732, -6.23..."
31356,138475,"[62376, 85056, 12, 1438, 2082, 798, 57353, 15,...","[-6.6155571937561035, -6.523186683654785, -6.4..."
31357,138486,"[26939, 5016, 66200, 48342, 31956, 28, 1365, 1...","[-7.121240139007568, -7.089005947113037, -6.49..."
31358,138492,"[5612, 52283, 54771, 51927, 31427, 3113, 47894...","[-6.9151997566223145, -6.4519219398498535, -6...."


In [51]:
# len(
#     tttmp[tttmp['userID']==11]['prediction']
# )
tttmp[tttmp['userID']==11]['prediction'].values[0][-10:]

[5.27623987197876,
 5.322011947631836,
 5.36540412902832,
 5.414576053619385,
 5.418984413146973,
 5.502008438110352,
 5.890142440795898,
 5.9291157722473145,
 6.114536762237549,
 7.077886581420898]

In [16]:
# submission = pd.DataFrame()
u_ll=[]
i_ll=[]
for idx,row in tttmp.iterrows():
    i_ll.append(row['itemID'][-10:])
    u_ll.append(row['userID'])

In [17]:
print(len(u_ll)==len(i_ll))

True


In [18]:
last_pd=pd.DataFrame(
    data={
        "userID": u_ll,
        "itemID": i_ll,
    }
)
submission = last_pd[["userID", "itemID"]].explode(
    "itemID", ignore_index=True
)
submission.to_csv("/opt/ml/Recommenders/bpr-epoch800-submission.csv",index=False)


In [22]:
all_predictions.head()

,userID,itemID,prediction
5154471,11,8961,5.365404
5154472,11,1396,-0.399799
5154473,11,471,-1.592354
5154474,11,1042,1.220800
5154475,11,1947,0.336741


In [21]:
k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

KeyboardInterrupt: 

In [ ]:
# Record results with papermill for tests
sb.glue("map", eval_map)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)

## References

1. Rendle, S., Freudenthaler, C., Gantner, Z., & Schmidt-Thieme, L. (2009, June). BPR: Bayesian personalized ranking from implicit feedback. https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf
2. Pan, R., Zhou, Y., Cao, B., Liu, N. N., Lukose, R., Scholz, M., & Yang, Q. (2008, December). One-class collaborative filtering. https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/04781145.pdf
3. **Cornac** - A Comparative Framework for Multimodal Recommender Systems. https://cornac.preferred.ai/